In [1]:
import pandas as pd
import os


In [6]:
corpus = "movie_corpus"
corpus_name = "movie_corpus"
datafile = os.path.join("..", "data", corpus,  "formatted_movie_lines.txt")
datafile
with open(datafile, "rb") as file:
    lines = file.readlines()
    for line in lines[:10]:
        print(str(line), "\n")
    


b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n" 

b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\n" 

b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n" 

b"You're asking me out.  That's so cute. What's your name again?\tForget it.\n" 

b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\n" 

b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\n" 

b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a date easy enough...\n" 

b'Why?\tUnsolved myster

b"Come to 45 Broadway.  Don't bring the Police.  Come alone or you'll be in my next film.\tLook asshole.  I've been threatened by better than you.\n" 

b"Look asshole.  I've been threatened by better than you.\tNo.  I'm the best that's ever threatened you.\n" 

b"No.  I'm the best that's ever threatened you.\tI'll meet you on one condition - I get exclusivity and you surrender to me.\n" 

b"I'll meet you on one condition - I get exclusivity and you surrender to me.\tWe'll talk about that.  Four o'clock gives you time to go to bank.  Three hundred thousand dollars.\n" 

b"We'll talk about that.  Four o'clock gives you time to go to bank.  Three hundred thousand dollars.\tWhat?  It doesn't work that way.\n" 

b"What?  It doesn't work that way.\tIf you don't want my film - I'll call another show.  And they will show it.\n" 

b"If you don't want my film - I'll call another show.  And they will show it.\tWait a minute.  Wait a minute.\n" 

b"Wait a minute.  Wait a minute.\tCome alone.  Brin

b'He was the only one of my bunch that was my friend... He was loyal, went all the way for you...\tIn all due respect, he sounds kind of pathetic to me.  The kind of guy that runs home to his momma or some girlfriend.  Have you two ace detectives checked that out?\n' 

b'Hey...\tShut up.\n' 

b"Shut up.\tWhat the hell's wrong?  I didn't do anything.\n" 

b"What do you want?  What's goin' on?\tShut up.\n" 

b'Stall.\tWhat do you want?\n' 

b"Keep stallin'.\tAlright, I'm coming...hold on.\n" 

b"How hot are they?\tHot?  Hey, they're not even room temperature.\n" 

b"How ya doin'?\tCan't complain.\n" 

b"Can't complain.\tWe got a lot to talk about.\n" 

b'We got a lot to talk about.\tYeah, old times.\n' 

b"Yeah, old times.\tWe'll follow you.  Take it slow,okay?\n" 

b"We'll follow you.  Take it slow,okay?\tSure, right.\n" 

b'Surprise, Luther.\tWhaddya want?  I thought you were locked up-\n' 

b'Whaddya want?  I thought you were locked up-\tI want the money, asshole, what do you think?  

b'They brought her into the room... one man, he put a knife to her throat and he raped her...\tNo...\n' 

b'No...\tHe raped her and...and...and he murdered her...he cut her up with knifes...\n' 

b'He raped her and...and...and he murdered her...he cut her up with knifes...\tNo... no... no...\n' 

b'No... no... no...\tThey killed her, and they took her out in the forest somewhere and they buried her...\n' 

b'They killed her, and they took her out in the forest somewhere and they buried her...\tWhy... why are you doing this to me... ?\n' 

b"Why... why are you doing this to me... ?\tThey murdered her, Mrs. Mathews, I'm sorry.  It happened a month after she ran away.  She's been dead all this time...\n" 

b"Yes... I remember Mary\tYou... you do?  You're sure? Please, Sister, will you take another look, make sure...\n" 

b"You... you do?  You're sure? Please, Sister, will you take another look, make sure...\tYes.  I remember her.\n" 

b"Do you know what happened to her?\tI'm trying to fin


b"There's a furnace.\tYou're not using it today?\n" 

b"You're not using it today?\tIt's broke I guess. There's an electric in the bedroom.\n" 

b"It's broke I guess. There's an electric in the bedroom.\tMaybe Wade should take a look at it. Your pipes'll freeze.  Wade, would you do that?\n" 

b"Coffee's perked.\tWhen did she die?\n" 

b"When did she die?\tIs...? She's dead then?\n" 

b'It makes me sad.\tCan --?\n' 

b'Can --?\tMakes me sad it was her. Instead of me. I shoulda froze.\n' 

b"Whitehouse. Next time, phone ahead.\tHow's that?\n" 

b"I said, 'Next time, phone ahead.'\tJesus Christ. Mr. Gordon, when I come all the way to serve somebody a summons, I don't call ahead for an appointment.\n" 

b"Jesus Christ. Mr. Gordon, when I come all the way to serve somebody a summons, I don't call ahead for an appointment.\tWhat the hell are you talking about?\n" 

b"What the hell are you talking about?\tI'm issuing you a ticket. Moving violation.\n" 

b"I'm issuing you a ticket. Moving vio

...

b"It is new, it is, isn't it, Sire?\tYes, indeed.\n" 

b'Yes, indeed.\tAnd German?\n' 

b'And German?\tOh, yes. Absolutely. German. Unquestionably!\n' 

b'Oh, yes. Absolutely. German. Unquestionably!\tSo then you like it? You really like it, Your Majesty?\n' 

b"So then you like it? You really like it, Your Majesty?\tOf course I do. It's very good. Of course now and then - just now and then - it gets a touch elaborate.\n" 

b"Of course I do. It's very good. Of course now and then - just now and then - it gets a touch elaborate.\tWhat do you mean, Sire?\n" 

b'What do you mean, Sire?\tWell, I mean occasionally it seems to have, how shall one say?  How shall one say, Director?\n' 

b"I don't understand. There are just as many notes, Majesty, as are required. Neither more nor less.\tMy dear fellow, there are in fact only so many notes the ear can hear in the course of an evening. I think I'm right in saying that, aren't I, Court Composer?\n" 

b"My dear, young man, don't take it too hard.

b"I'm talking about a girl you want to fuck, so give me a break.\tWell, anyway, do you think she'll be there?\n" 

b"Well, anyway, do you think she'll be there?\tI don't know.\n" 

b"I don't know.\tRendezvous in Rome starring Jack Goodman and Debbie Klein. The love affair that shocked Europe! See torrid lovemaking at its most explicit! See Jack and Debbie expose their lust in the sacred halls of the Vatican! Never has the screen dared...\n" 

b"Rendezvous in Rome starring Jack Goodman and Debbie Klein. The love affair that shocked Europe! See torrid lovemaking at its most explicit! See Jack and Debbie expose their lust in the sacred halls of the Vatican! Never has the screen dared...\tIf you don't stop, I'm going to kill you.\n" 

b"If you don't stop, I'm going to kill you.\tI have to make love to her. It's very simple. She has no choice really.\n" 

b"I have to make love to her. It's very simple. She has no choice really.\tIt just fascinates me that you can spend so much energy on som


b"Boys...\tGo home, Doc.  They ain't gonna hang no more Liberty boys.\n" 

b"What do you say, sir?\tGo on. You're pretty much all healed up.\n" 

b"You know you're welcome any time!\tYesss, but I was thinking, I could come by, and then take Zee out. Some place near. With other folk. Near. Here.  But out.\n" 

b"Yesss, but I was thinking, I could come by, and then take Zee out. Some place near. With other folk. Near. Here.  But out.\tIt's fine by me, Jesse.\n" 

b"Daddy, don't start with this again.\tZerelda, it's no coincidence. The railroad men come through, offering to buy up land. Nobody sells. Then they start hanging men who own farms for treason?\n" 

b"He's going to be fine, right Daddy?\tThe bullet came out clean, but he lost a whole lot of blood. Praying wouldn't hurt.\n" 

b"They're gone. What are you --\tI fooled them into thinking I was alone.\n" 

b'I fooled them into thinking I was alone.\tWell, I hope the boy pulls through. We should know in the morning.\n' 

b"Well, I h


b'Go where? You can\'t get away from people like this.\t"""Like this?"" It\'s Gary you\'re talking about."\n' 

b'"""Like this?"" It\'s Gary you\'re talking about."\tYou think I don\'t know that?\n' 

b"You think I don't know that?\tMilo. Why would he --\n" 

b'Milo. Why would he --\t"How should I know? ""Solving a problem,"" I guess. Or needing to control everything. I don\'t know.  I\'ve gotta get in there."\n' 

b'"How should I know? ""Solving a problem,"" I guess. Or needing to control everything. I don\'t know.  I\'ve gotta get in there."\tEven if all this were true. There\'re 20 other buildings. All of them filled with computers and --\n' 

b"Even if all this were true. There're 20 other buildings. All of them filled with computers and --\tIt's the only one with dishes on the roof. The studio's a front. That's why they keep postponing its opening.  ...gotta get in there.\n" 

b"It's the only one with dishes on the roof. The studio's a front. That's why they keep postponing its o

KeyboardInterrupt: 

In [8]:
# building vocabulary

In [ ]:
PAD_token = 0
SOS_token = 1
EOS_token = 2

class Vocabulary():
    def __init__(self, name):
        self.name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {
            PAD_token: "PAD",
            SOS_token: "SOS",
            EOS_token: "EOS",
        }
        self.num_words = 3
        
    def addWord(self, w):
        if w not in self.word2index:
            self.word2index[w] = self.num_words
            self.word2count[w] = 1
            self.index2word[self.num_words] = w
            
            self.num_words += 1
        else:
            self.word2count[w] += 1
            
            
    def addSentence(self, sent):
        for word in sent.split(" "):
            self.addWord(word)
            
    def trim(self, min_cnt):
        if self.trimmed:
            return
        self.trimmed = True
        words_to_keep = []
        for k, v in self.word2count.items():
            if v >=  min_cnt:
                words_to_keep.append(k)
                
        # re build       
        self.word2index = {}
        self.word2count = {}
        self.index2word = {
            PAD_token: "PAD",
            SOS_token: "SOS",
            EOS_token: "EOS",
        }
        self.num_words = 3
        
        for w in words_to_keep:
            self.addWord(word)
            
            
            